In [5]:
import docker
import subprocess

client = docker.from_env()
running_containers = client.containers.list()
sharedir = "share" 

for cont in running_containers:
    if "omero" and "server" in cont.name:
        print(cont.name)
        ome_server = cont.name
        break
        
subprocess.call(["docker", "exec", ome_server, "/opt/omero/server/OMERO.server/bin/omero", "-s" "localhost", "-u", "root", "login", "-w", "omero"])
test = subprocess.check_output(["docker", "exec", ome_server, "/opt/omero/server/OMERO.server/bin/omero", "obj", "new", "Dataset", "name=annotation_dummies"])
test = test.decode().strip()
subprocess.call(["docker", "exec", ome_server, "/opt/omero/server/OMERO.server/bin/omero", "import", "-d", test, "--transfer=ln_s", "/OMERO/{}".format(sharedir)])

ome-test-omero-server


0

In [87]:
from omero.gateway import BlitzGateway, MapAnnotationWrapper
from omero.model import EllipseI
import omero


class OmeAnnotExtractor():
    """
        Connection Manager and Methods for extracting annotations from an Omero Host
        :param ome_server: Host name
        :param user: User
        :param pw: Password
    """
    
    def __init__(self, user, pw, ome_server):
        self.user = user
        self.pw = pw
        self.server = ome_server
        self.conn = None
    
    
    def connect(self):
        """
            Connect to an OMERO server
        """
        self.conn = BlitzGateway(self.user, self.pw,
                            host=self.server, secure=False)
        self.conn.connect()
        self.conn.c.enableKeepAlive(60)
        
        return


    def disconnect(self):
        """
            Disconnect from an OMERO server
            :param conn: The BlitzGateway
        """
        assert self.conn is not None, "Connection not established please use "
        self.conn.close()
    
    def list_projects(self):
        
        """
            List the projects available to the current user
        """
        my_exp_id = self.conn.getUser().getId()
        default_group_id = self.conn.getEventContext().groupId
        projects =  self.conn.getObjects("Project", opts={'owner': my_exp_id,
                                            'group': default_group_id,
                                            'order_by': 'lower(obj.name)',
                                            'limit': 5, 'offset': 0})
        for project in projects:
            print(project.name)
        
        return projects
    
    def select_project(self, sel_project):
        projects = self.list_projects()
        
    def list_object_properties(self, ome_object):
        """
            List the callable members of the passed OME Object
            :param ome_object: OME Model object
        """
        
        object_methods = [method_name for method_name in dir(ome_object)
                    if callable(getattr(ome_object, method_name))]
        print(object_methods)
    
    def list_datasets(self, project):
        """
            List the projects available to the current user
            :param projects: List of projects
        """
        dataset_dict = {}
        for d in ome_parse.conn.getObjects("Dataset"):
            dataset_dict.update({d.id:[d.name]})
    
    def list_images(self):
        pass
        
    def retrieve_geometry(self, imageId):
        roi_service = self.conn.getRoiService()
        result = roi_service.findByImage(imageId, None)
        
        for roi in result.rois:
            print("ROI:  ID:", roi.getId().getValue())
            #print(self.list_object_properties(roi))
            #print(roi.getDetails())
            #print(self.list_object_properties(roi.getDetails()))
            for s in roi.copyShapes():
                shape = {}
                shape['id'] = s.getId().getValue()
                shape['theT'] = s.getTheT().getValue()
                shape['theZ'] = s.getTheZ().getValue()
                if s.getTextValue():
                    shape['textValue'] = s.getTextValue().getValue()
                if type(s) == omero.model.RectangleI:
                    shape['type'] = 'Rectangle'
                    shape['x'] = s.getX().getValue()
                    shape['y'] = s.getY().getValue()
                    shape['width'] = s.getWidth().getValue()
                    shape['height'] = s.getHeight().getValue()
                    #print(self.list_object_properties(s))
                    #print(s.getVersion)
                    print(self.list_object_properties(s.getDetails().getCreationEvent))
                elif type(s) == omero.model.EllipseI:
                    shape['type'] = 'Ellipse'
                    shape['x'] = s.getX().getValue()
                    shape['y'] = s.getY().getValue()
                    shape['radiusX'] = s.getRadiusX().getValue()
                    shape['radiusY'] = s.getRadiusY().getValue()
                elif type(s) == omero.model.PointI:
                    shape['type'] = 'Point'
                    shape['x'] = s.getX().getValue()
                    shape['y'] = s.getY().getValue()
                elif type(s) == omero.model.LineI:
                    shape['type'] = 'Line'
                    shape['x1'] = s.getX1().getValue()
                    shape['x2'] = s.getX2().getValue()
                    shape['y1'] = s.getY1().getValue()
                    shape['y2'] = s.getY2().getValue()
                elif type(s) == omero.model.MaskI:
                    shape['type'] = 'Mask'
                    shape['x'] = s.getX().getValue()
                    shape['y'] = s.getY().getValue()
                    shape['width'] = s.getWidth().getValue()
                    shape['height'] = s.getHeight().getValue()
                elif type(s) == omero.model.PolygonI:
                    shape['type'] = 'Polygon'
                    self.list_object_properties(s)
                elif type(s) in (omero.model.LabelI):
                    print(type(s), " Not supported by this code")
                # Do some processing here, or just print:
                print("   Shape:",)
                for key, value in shape.items():
                    print("  ", key, value,)
                print("")

In [88]:
import uuid
from inspect import ismethod

ome_parse = OmeAnnotExtractor("root", "omero", ome_server)
ome_parse.connect()
annot_dict = {}

for d in ome_parse.conn.getObjects("Dataset"):
    for image in d.listChildren():
        if "macro" in image.getName() or "label" in image.getName():
            continue
        #ome_parse.list_object_properties(image)
        slide_id = "MUG-GRAZ_SLIDE-" + str(uuid.uuid4())
        annot_dict["slide_id"] = slide_id
        annot_dict["metdata"] = {"scan":{}, "acq_date":image.getDate()}
        annot_filename = image.getName() + "annotations.json"
        ome_parse.retrieve_geometry(image.id)
        
        
ome_parse.disconnect()

ROI:  ID: 1
   Shape:
   id 1
   theT 0
   theZ 0
   type Ellipse
   x 11590.36996059412
   y 5189.527584116399
   radiusX 1436.576841467113
   radiusY 459.02455289481713

ROI:  ID: 2
<bound method DetailsI.getCreationEvent of object #0 (::omero::model::Details)
{
    _owner = object #1 (::omero::model::Experimenter)
    {
        _id = object #2 (::omero::RLong)
        {
            _val = 0
        }
        _details = <nil>
        _loaded = False
        _version = <nil>
        _groupExperimenterMapSeq = 
        {
        }
        _groupExperimenterMapLoaded = False
        _omeName = <nil>
        _firstName = <nil>
        _middleName = <nil>
        _lastName = <nil>
        _institution = <nil>
        _ldap = <nil>
        _email = <nil>
        _config = 
        {
        }
        _annotationLinksSeq = 
        {
        }
        _annotationLinksLoaded = False
        _annotationLinksCountPerOwner = 
        {
        }
    }
    _group = object #3 (::omero::model::Exp

In [30]:
import pandas as pd